# Use Rootly with agents.json
This notebook demonstrates how to use agents.json to interact with the Rootly API. This simple example shows loading the agents.json file and using it to execute Rootly tasks.


In [1]:
pip list

Package                   Version     Editable project location
------------------------- ----------- -------------------------------------------------------
agentsjson                0.1.1       /Users/yagnyapatel/Projects/Wildcard/agents-json/python
aiohappyeyeballs          2.4.4
aiohttp                   3.11.11
aiosignal                 1.3.2
alembic                   1.14.1
annotated-types           0.7.0
anyio                     4.8.0
appnope                   0.1.4
asttokens                 3.0.0
asyncpg                   0.30.0
attrs                     25.1.0
babel                     2.17.0
bcrypt                    4.2.1
cachetools                5.5.1
certifi                   2024.12.14
cffi                      1.17.1
charset-normalizer        3.4.1
click                     8.1.8
comm                      0.2.2
cryptography              44.0.0
curlify                   2.2.1
debugpy                   1.8.12
decorator                 5.1.1
Delorean                  1.0.

In [2]:
%pip install agentsjson
%pip install openai


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Setup API keys and references to `agents.json` files

In [9]:
ROOTLY_API_KEY="your_rootly_api_key"
OPENAI_API_KEY="your_openai_api_key"
agents_json_url = "https://gist.githubusercontent.com/yagnyaPatel/f21f8c0a6c221213584bbb7a99a0e1dc/raw/ec1cc50a243a1e63c161fc3b9a27b6302dac252b/agents.json"

### Load the agents.json file

In [10]:
from agentsjson.core.models import Flow
from agentsjson.core.models.bundle import Bundle
import agentsjson.core as core

# load the agents.json file
data: Bundle = core.load_agents_json(agents_json_url)
flows = data.agentsJson.flows

### Define the Agent

In [15]:
query = "Get me the incident details on the monkey in the server room"

In [16]:
from agentsjson.core import ToolFormat

# Format the flows data for the prompt
flows_context = core.flows_prompt(flows)

# Create the system prompt
system_prompt = f"""You are an AI assistant that helps users interact with the Rootly API.
You have access to the following API flows:

{flows_context}

Analyze the user's request and use the appropriate API flows to accomplish the task.
You must give your arguments for the tool call as Structued Outputs JSON with keys `parameters` and `requestBody`"""

### Configure Auth

In [17]:
from agentsjson.core.models.auth import AuthType, BearerAuthConfig
auth = BearerAuthConfig(type=AuthType.BEARER, token=ROOTLY_API_KEY)

### Run the agent

In [18]:
from openai import OpenAI
from agentsjson.core.executor import execute_flows
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ],
    tools=core.flows_tools(flows, format=ToolFormat.OPENAI),
    temperature=0
)

response = execute_flows(response, format=core.ToolFormat.OPENAI, bundle=data, flows=flows, auth=auth)

response

{'retrieve_top_incident_details': {'success': {'data': {'id': '41b5b7f4-db7d-4a69-a44f-13c34ab20af9', 'type': 'incidents', 'attributes': {'parent_incident_id': None, 'sequential_id': 3, 'title': 'Prod Down', 'slug': 'prod-down', 'kind': 'normal', 'private': False, 'summary': 'The monkey got into the server room and took prod down', 'status': 'started', 'source': 'web', 'url': 'https://rootly.com/account/incidents/3-prod-down', 'short_url': 'https://root.ly/rjaxdj', 'user': {'data': {'id': '86651', 'type': 'users', 'attributes': {'name': 'Hunly AI', 'email': 'contact@hunly.io', 'phone': None, 'phone_2': None, 'full_name': 'Hunly AI', 'full_name_with_team': '[Hunly] Hunly AI', 'slack_id': None, 'time_zone': 'America/Los_Angeles', 'updated_at': '2025-02-13T15:32:22.429-08:00', 'created_at': '2025-02-13T02:34:09.838-08:00'}, 'relationships': {'email_addresses': {'data': [{'id': '705e87e1-e7b9-4e3c-ade3-387c353ae5f2', 'type': 'user_email_addresses'}]}, 'phone_numbers': {'data': []}, 'device